# Creating a network from a file

In [ ]:
import getpass

from roseau.load_flow import *

## From JSON

The first format that can be read is the JSON format. The `from_json` class-method of `ElectricalNetwork` can be used
for that.

In [ ]:
en = ElectricalNetwork.from_json("data/LVFeeder00939_summer.json")

In [ ]:
# Authentication
auth = (input("Please enter your username:"), getpass.getpass("Please enter your password:"))

Then, a load flow can be directly solved.

In [ ]:
en.solve_load_flow(auth=auth)

The results are accessible the same way as explained in the [Getting Started](Getting_Started.ipynb) page

In [ ]:
en.res_buses.head(n=8)

In [ ]:
en.res_branches.head(n=8)

Note that the buses, branches (lines, transformers and switches), and loads can be accessed from the electrical network
using the `buses`, `branches` and `loads` fields.

In [ ]:
en.buses

## Adding elements

It is possible to add new buses, loads, lines, switches, transformers to an existing network
by simply creating the elements and linking them to an element already in the network. For
example, the code below adds a new line and a constant power load.

In [ ]:
# Connect a new bus to the network
ground = en.grounds["ground"]
bus = Bus(id="new_bus", phases="abcn")
load = PowerLoad(id="new_load", bus=bus, phases="abcn", powers=[100, 0, 0])
lp = LineParameters.from_name_lv("S_AL_240")
line = Line(
    id="new_line",
    bus1=en.buses["LVBus004111"],
    bus2=bus,
    phases="abcn",
    ground=ground,
    parameters=lp,
    length=0.5,
)

Then, the load flow can be solved again, and the new results are available.

In [ ]:
# Rerun the results
en.solve_load_flow(auth=auth)
en.res_buses.head(n=8)

In [ ]:
en.res_branches.head(n=8)

## Saving the modified network

The modified network can be saved using the `en.to_json` method.

```python
en.to_json('new_network.json')
```